In [2]:
pip install warnings

warnings.filterwarnings('ignore')
import yfinance as yf

# Tickers and date range
tickers = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'V']
start_date = '2020-01-01'
end_date = '2025-09-24' 


data = yf.download(tickers, start=start_date, end=end_date)['Close']


data.to_csv('stock_prices.csv')

print("Data saved successfully to stock_prices.csv")

NameError: name 'warnings' is not defined

In [ ]:
import pandas as pd
portfolio_data= pd.read_csv("stock_prices.csv", index_col='Date', parse_dates=True)
display(portfolio_data)

In [ ]:
import numpy as np
daily_returns = np.log(portfolio_data / portfolio_data.shift(1))
expected_annual_returns = daily_returns.mean() * 252
annual_cov_matrix = daily_returns.cov() * 252
print (annual_cov_matrix)

In [ ]:
def portfolio_performance(weights, expected_returns, annual_cov_matrix, risk_free_rate):
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(annual_cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    return -sharpe_ratio

In [ ]:
from scipy.optimize import minimize

# Each weight between 0 and 1
num_assets = len(expected_annual_returns)
bounds = tuple((0, 0.5) for _ in range(num_assets))

# Sum of weights is 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

In [ ]:

    num_assets = len(expected_annual_returns)
    initial_weights = np.array(num_assets * [1. / num_assets])
    print(initial_weights)
    
    # We will assume the risk free rate is 0.03%
    risk_free_rate = 0.03
    
    
    result = minimize( fun=portfolio_performance,x0=initial_weights, args=(expected_annual_returns, annual_cov_matrix, risk_free_rate),method='SLSQP',
                      bounds=bounds,
                      constraints=constraints
    )

In [ ]:
# Extract the optimal weights from the result
optimal_weights = result.x

# Print the optimal allocation for each stock
print("Optimal Portfolio Allocation:")
for ticker, weight in zip(portfolio_data.columns, optimal_weights):
    print(f"- {ticker}: {weight*100:.2f}%")

# Calculate and display the performance of the optimal portfolio
optimal_return = np.sum(expected_annual_returns * optimal_weights)
optimal_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(annual_cov_matrix, optimal_weights)))
optimal_sharpe_ratio = (optimal_return - risk_free_rate) / optimal_volatility

print("\n--- Optimal Portfolio Performance ---")
print(f"Expected Annual Return: {optimal_return*100:.2f}%")
print(f"Annual Volatility (Risk): {optimal_volatility*100:.2f}%")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.2f}")

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(8, 8))
plt.pie(optimal_weights, labels=portfolio_data.columns, autopct='%1.1f%%', startangle=140)
plt.title('Optimal Portfolio Allocation')
plt.axis('equal')  


plt.savefig('optimal_portfolio_pie_chart.png')